In [1]:
import os
import sys
import time

import matplotlib.pyplot as plt
import numpy as np

import GCode
import GRBL


# Draw a Brick Pattern

Attempt at programmatically making a brick pattern.

All units in mm. ```1``` = ```1 mm```.

"Napkin" scratches.

![](http://luckofthedraw.fun/.imgs/brick_0003.jpeg)

Drawn by hand. ~18mm brick height.

![](http://luckofthedraw.fun/.imgs/brick_0001.jpeg)

> Standard bricks. The standard co-ordinating size for brickwork is 225 mm x 112.5 mm x 75 mm (length x depth x height). This includes 10 mm mortar joints, and so the standard size for a brick itself is 215 mm x 102.5 mm x 65 mm (length x depth x height).

In [2]:
# Standard brick dimensions.
BrickHeight = 65  # [mm]
BrickLength = 225  # [mm]
BrickDepth = 12.5  # [mm]
BrickRatio = 215 / 65  # [dimensionless]


In [3]:
# Poplar 1x4". Cut
BlockHeight = 89.0  # mm
BlockLength = 2 * BlockHeight  # mm


In [4]:
# Drawing configuration.
# How many rows of bricks to draw on the block.
N_BrickRows = 5  # [dimensionless]

# Dimensions of a 'brick' projected onto the block of wood.
H_Block_Brick = BlockHeight / N_BrickRows  # [mm]
L_Block_Brick = H_Block_Brick * BrickRatio  # [mm]


# Code:

In [5]:
flip = np.array([[1, 1], [1, 0]])
transform_tuple = (
    np.eye(2),  # Identity matrix, do nothing.
    np.eye(2),  # Do nothing, for debugging.
    flip,  # Flip the matrix, reduces travel time.
)
vertical_brick_lines_tuple = (
    np.arange(L_Block_Brick, BlockLength, L_Block_Brick),# Odd rows.
    np.arange(L_Block_Brick / 2, BlockLength, L_Block_Brick), # Even rows.
)
horizontal_brick_lines = np.linspace(0, BlockHeight, N_BrickRows, endpoint=False)

Lines parallel to the X-axis.
Separates rows of bricks.

In [6]:
horizontal_brick_lines

array([ 0. , 17.8, 35.6, 53.4, 71.2])

Lines parallel to the Y-axis.

In [7]:
vertical_brick_lines_tuple

(array([ 58.87692308, 117.75384615, 176.63076923]),
 array([ 29.43846154,  88.31538462, 147.19230769]))

In [8]:
BlockBrick=GCode.Program()
BlockBrick.lines=list()

for idx in range(1, len(horizontal_brick_lines)):
    # Top horizontal line that defines each 'brick'
    horizontal_brick_line = horizontal_brick_lines[idx]
    row_line_points = np.array(
        [[0, horizontal_brick_line], [BlockLength, horizontal_brick_line]]
    )
    # Transform to perform on the row points.
    transform = transform_tuple[np.mod(idx, 2)]
    
    row_line_points = np.matmul(transform, row_line_points)
    line_ = GCode.Line(
        points=row_line_points,
    )
    
    BlockBrick.lines.append(line_)
    
    # Vertical brick line.
    vertical_brick_lines = vertical_brick_lines_tuple[np.mod(idx, len(vertical_brick_lines_tuple))]
    
    start_point_y = horizontal_brick_lines[idx - 1]
    end_point_y = horizontal_brick_lines[idx]
    
    for idx2, vertical_brick_line in enumerate(vertical_brick_lines):
        transform = transform_tuple[np.mod(idx2, 2)]
        column_line_points = (
            np.array(
                [
                    [vertical_brick_line, start_point_y],
                    [vertical_brick_line, end_point_y],
                ]
            ),
        )
        column_line_points = np.matmul(transform, column_line_points)
        line_ = GCode.Line(column_line_points)
        BlockBrick.lines.append(line_)

In [9]:
BlockBrick.lines

[Line<cut=178.0mm, F=300, S=150>,
 Line<cut=17.8mm, F=300, S=150>,
 Line<cut=17.8mm, F=300, S=150>,
 Line<cut=17.8mm, F=300, S=150>,
 Line<cut=178.0mm, F=300, S=150>,
 Line<cut=17.8mm, F=300, S=150>,
 Line<cut=17.8mm, F=300, S=150>,
 Line<cut=17.8mm, F=300, S=150>,
 Line<cut=178.0mm, F=300, S=150>,
 Line<cut=17.8mm, F=300, S=150>,
 Line<cut=17.8mm, F=300, S=150>,
 Line<cut=17.8mm, F=300, S=150>,
 Line<cut=178.0mm, F=300, S=150>,
 Line<cut=17.8mm, F=300, S=150>,
 Line<cut=17.8mm, F=300, S=150>,
 Line<cut=17.8mm, F=300, S=150>]

In [10]:
BlockBrick.time

1033.47291050604

In [11]:
BlockBrick.dist

2622.30582101208

In [12]:
BlockBrick.generate_gcode()

In [13]:
BlockBrick

Program<lines=16>

In [14]:
cnc = GRBL.GRBL(port="/dev/cnc_3018")
cnc.reset()

(0, ['ok', '', "Grbl 1.1f ['$' for help]", 'ok'])

In [15]:
cnc.home()

14

In [16]:
BlockBrick.machine=cnc

In [17]:
BlockBrick.run()